In [2]:
import requests
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings'

In [99]:
match_stages = load_csv_dataset('create_dataset_1/CSVs we use often/match_stages.csv')

In [4]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')
latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
pd.set_option('display.max_columns', None)

In [23]:
from datetime import datetime

def compare_dates(date1, date2):
    # Convert strings to datetime objects
    date1_obj = datetime.strptime(date1, '%Y-%m-%d')
    date2_obj = datetime.strptime(date2, '%Y-%m-%d')
    
    # Compare dates
    if date1_obj < date2_obj:
        return -1
    elif date1_obj > date2_obj:
        return 1
    else:
        return 0

In [28]:
row = latest_match_data.iloc[0]
date = row['Date']
competition = row['Competition']
season = row['Season']

#get the match stages for the date and competition
match_stages[(match_stages['Competition'] == competition) & (match_stages['Year'] == season)]#.iloc[0]['Date Start']

,Stage,Date Start,Date End,Competition,Year
0,Preliminary Round 1,2011-06-29,2011-07-03,"World Cup Qualifiers, Asia",2011
1,Preliminary Round 2,2011-07-23,2011-07-28,"World Cup Qualifiers, Asia",2011
2,Group Stage,2011-09-02,2012-02-29,"World Cup Qualifiers, Asia",2011


In [ ]:
#given the date of a match

    #KNOCKOUT STAGE CHECK
        #if it's between the start date and the end date of the following stages:
            #Finals
            #Semi-Finals
            #Quarter-Finals
            #Final
            #Playoff Round
            #Round of 16
        #both teams are in survival mode 


In [27]:
compare_dates(date, match_stages[(match_stages['Competition'] == competition) & (match_stages['Year'] == season)].iloc[0]['Date Start'])


1

In [100]:
from datetime import datetime

def count_matches_in_date_range(latest_match_data, start_date, end_date):
    # Convert string dates to datetime objects
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Filter matches within the date range
    matches_in_range = latest_match_data[
        (latest_match_data['Date'] >= start_date) & (latest_match_data['Date'] <= end_date)
    ]
    
    # Count matches for each team
    team_match_counts = {}
    for index, row in matches_in_range.iterrows():
        teams = [row['Team.1.Name'], row['Team.2.Name']]
        for team in teams:
            if team in team_match_counts:
                team_match_counts[team] += 1
            else:
                team_match_counts[team] = 1
    
    return team_match_counts

def analyze_team_match_counts(team_match_counts):
    match_counts = list(team_match_counts.values())
    unique_counts = set(match_counts)
    
    if len(unique_counts) == 1:
        return f"All teams played {unique_counts.pop()} games."
    else:
        return f"Different match counts: {unique_counts}"

# Example usage
start_date = '2011-06-29'
end_date = '2011-07-03'
team_match_counts = count_matches_in_date_range(latest_match_data, start_date, end_date)
result = analyze_team_match_counts(team_match_counts)

In [101]:
import pandas as pd
from datetime import datetime

def count_matches_in_date_range(latest_match_data, start_date, end_date):
    # Convert string dates to datetime objects
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Filter matches within the date range
    matches_in_range = latest_match_data[
        (latest_match_data['Date'] >= start_date_obj) & (latest_match_data['Date'] <= end_date_obj)
    ]
    
    # Count matches for each team
    team_match_counts = {}
    for index, row in matches_in_range.iterrows():
        teams = [row['Team.1.Name'], row['Team.2.Name']]
        for team in teams:
            if team in team_match_counts:
                team_match_counts[team] += 1
            else:
                team_match_counts[team] = 1
    
    return team_match_counts, matches_in_range

def analyze_team_match_counts(team_match_counts):
    match_counts = list(team_match_counts.values())
    unique_counts = set(match_counts)
    
    if len(unique_counts) == 1:
        return unique_counts.pop()
    else:
        return unique_counts

# Initialize a list to store the results
results = []

non_knockout_stages = match_stages[~match_stages['Stage'].isin(['Finals', 'Semi-Finals', 'Quarter-Finals', 'Final', 'Playoff Round', 'Round of 16'])]

for index, stage in non_knockout_stages.iterrows():
    start_date = stage['Date Start']
    end_date = stage['Date End']
    competition = stage['Competition']
    
    # Filter matches for the specific competition
    competition_matches = latest_match_data[latest_match_data['Competition'] == competition]
    
    # Get match counts for the date range
    team_match_counts, matches_in_range = count_matches_in_date_range(competition_matches, start_date, end_date)
    result = analyze_team_match_counts(team_match_counts)
    
    # Format the result
    if isinstance(result, set):
        result = ', '.join(map(str, result))
    elif result == 0:
        result = 0
    
    # Store the result in a dictionary
    stage_result = {
        "Competition": competition,
        "Stage": stage['Stage'],
        "Start Date": start_date,
        "End Date": end_date,
        "Match Counts": result
    }
    results.append(stage_result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


In [102]:
# Display rows where the "Match Counts" column is an empty set represented as a string
blank_match_counts_df = results_df[results_df['Match Counts'] == '']

# Display the filtered DataFrame
blank_match_counts_df

,Competition,Stage,Start Date,End Date,Match Counts
35,"World Cup Qualifiers, CONCACAF",Round 3,2022-06-04,2022-06-08,
45,"Euros, Qualifiers",Qualifying Round,2010-08-11,2011-11-15,
48,Euros,Group Stage,2012-06-08,2012-06-18,
51,Copa America,Group Stage,2011-07-01,2011-07-13,
56,Olympics,Group Stage,2012-07-26,2012-08-01,
61,Gold Cup,Group Stage,2011-06-05,2011-06-13,
67,"Africa Cup of Nations, Qualifiers",Qualifying Round,2010-07-01,2011-10-09,
68,"Africa Cup of Nations, Qualifiers",Qualifying Round,2012-01-01,2012-10-14,
73,Africa Cup of Nations,Group Stage,2012-01-21,2012-01-31,
74,Africa Cup of Nations,Group Stage,2013-01-19,2013-01-29,


In [132]:
# Filter rows where the length of the "Match Counts" column is more than 1
multiple_match_counts_df = results_df[results_df['Match Counts'].apply(lambda x: ',' in str(x))]
# multiple_match_counts_df = results_df[results_df['Match Counts'].apply(lambda x: len(str(x)) > 1)]

# Display the filtered DataFrame
multiple_match_counts_df

,Competition,Stage,Start Date,End Date,Match Counts
6,"World Cup Qualifiers, Asia",Preliminary Round 2,2015-06-11,2016-03-29,"8, 5, 6, 7"
8,"World Cup Qualifiers, Asia",Round 3,2017-09-05,2017-11-15,"1, 3"
9,"World Cup Qualifiers, Asia",Preliminary Round 1,2019-06-06,2019-06-11,"1, 2"
10,"World Cup Qualifiers, Asia",Preliminary Round 2,2019-09-05,2021-06-15,"8, 5, 7"
13,"World Cup Qualifiers, Africa",Preliminary Round,2011-11-11,2011-11-15,"1, 2"
16,"World Cup Qualifiers, Africa",Preliminary Round 1,2015-10-05,2015-10-13,"1, 2"
18,"World Cup Qualifiers, Africa",Group Stage,2016-10-07,2017-11-14,"6, 7"
19,"World Cup Qualifiers, Africa",Round 3,2017-08-31,2017-11-14,"4, 5"
23,"World Cup Qualifiers, CONCACAF",Preliminary Round,2011-06-15,2011-07-16,"1, 2"
24,"World Cup Qualifiers, CONCACAF",Group Stage,2011-09-02,2011-11-15,"3, 4, 5, 6"


In [133]:
import pandas as pd

# Define the function to count team appearances
def count_team_appearances(latest_match_data, competition, start_date, end_date):
    # Convert string dates to datetime objects
    start_date_obj = pd.to_datetime(start_date)
    end_date_obj = pd.to_datetime(end_date)

    # Convert the 'Date' column in latest_match_data to datetime objects
    latest_match_data['Date'] = pd.to_datetime(latest_match_data['Date'])

    # Filter latest_match_data
    filtered_matches = latest_match_data[
        (latest_match_data['Competition'] == competition) &
        (latest_match_data['Date'] >= start_date_obj) &
        (latest_match_data['Date'] <= end_date_obj)
    ]

    # Initialize a dictionary to count team appearances
    team_appearance_counts = {}

    # Iterate through the filtered matches and count appearances
    for index, row in filtered_matches.iterrows():
        teams = [row['Team.1.Name'], row['Team.2.Name']]
        for team in teams:
            if team in team_appearance_counts:
                team_appearance_counts[team] += 1
            else:
                team_appearance_counts[team] = 1

    return team_appearance_counts

# Apply the function to each row of multiple_match_counts_df
team_appearances_list = []

for index, row in multiple_match_counts_df.iterrows():
    competition = row['Competition']
    start_date = row['Start Date']
    end_date = row['End Date']
    
    team_appearances = count_team_appearances(latest_match_data, competition, start_date, end_date)
    
    # Store the result in a dictionary
    result = {
        "Competition": competition,
        "Stage": row['Stage'],
        "Start Date": start_date,
        "End Date": end_date,
        "Team Appearances": team_appearances
    }
    team_appearances_list.append(result)

# Convert the results to a DataFrame
team_appearances_df = pd.DataFrame(team_appearances_list)

# Display the DataFrame
team_appearances_df

,Competition,Stage,Start Date,End Date,Team Appearances
0,"World Cup Qualifiers, Asia",Preliminary Round 2,2015-06-11,2016-03-29,"{'Iraq': 6, 'Chinese Taipei': 6, 'Yemen': 8, 'Uzbekistan': 8, 'Bahrain': 8, 'Kuwait': 5, 'Lebanon': 8, 'Myanmar': 7, 'Malaysia': 8, 'Timor-Leste': 8, 'Saudi Arabia': 8, 'Palestine': 8, 'United Arab Emirates': 8, 'Bangladesh': 8, 'Kyrgyzstan': 8, 'Tajikistan': 8, 'Jordan': 8, 'Australia': 8, 'Hong Kong': 8, 'Bhutan': 8, 'Maldives': 8, 'Qatar': 8, 'China': 8, 'Guam': 8, 'Turkmenistan': 8, 'India': 8, 'Oman': 8, 'Iran': 8, 'Cambodia': 8, 'Singapore': 8, 'Afghanistan': 8, 'Syria': 8, 'Japan': 8, 'Thailand': 5, 'Vietnam': 5, 'Laos': 7, 'South Korea': 7, 'Philippines': 8, 'North Korea': 8}"
1,"World Cup Qualifiers, Asia",Round 3,2017-09-05,2017-11-15,"{'Syria': 3, 'Australia': 3, 'Iran': 1, 'Qatar': 1, 'China': 1, 'Uzbekistan': 1, 'South Korea': 1, 'Thailand': 1, 'Iraq': 1, 'United Arab Emirates': 1, 'Saudi Arabia': 1, 'Japan': 1}"
2,"World Cup Qualifiers, Asia",Preliminary Round 1,2019-06-06,2019-06-11,"{'Guam': 2, 'Bhutan': 2, 'Brunei': 2, 'Mongolia': 2, 'Timor-Leste': 2, 'Malaysia': 2, 'Bangladesh': 2, 'Laos': 2, 'Pakistan': 2, 'Cambodia': 2, 'Macau': 1, 'Sri Lanka': 1}"
3,"World Cup Qualifiers, Asia",Preliminary Round 2,2019-09-05,2021-06-15,"{'Guam': 8, 'Maldives': 8, 'Philippines': 8, 'Syria': 8, 'China': 8, 'Chinese Taipei': 8, 'Jordan': 8, 'Kuwait': 8, 'Nepal': 8, 'Australia': 8, 'Cambodia': 8, 'Hong Kong': 8, 'Bahrain': 8, 'Iraq': 8, 'Iran': 8, 'Singapore': 8, 'Yemen': 8, 'Palestine': 8, 'Uzbekistan': 8, 'Saudi Arabia': 8, 'India': 8, 'Oman': 8, 'Qatar': 8, 'Afghanistan': 8, 'Bangladesh': 8, 'Mongolia': 8, 'Myanmar': 8, 'Tajikistan': 8, 'Kyrgyzstan': 8, 'Japan': 8, 'Thailand': 8, 'Vietnam': 8, 'Indonesia': 8, 'Malaysia': 8, 'United Arab Emirates': 8, 'North Korea': 5, 'Lebanon': 8, 'Sri Lanka': 7, 'Turkmenistan': 7, 'South Korea': 7}"
4,"World Cup Qualifiers, Africa",Preliminary Round,2011-11-11,2011-11-15,"{'Burundi': 2, 'Lesotho': 2, 'Madagascar': 2, 'Equatorial Guinea': 2, 'Tanzania': 2, 'Chad': 2, 'Kenya': 2, 'Seychelles': 2, 'Rwanda': 2, 'Eritrea': 2, 'Congo': 2, 'São Tomé and Príncipe': 2, 'DR Congo': 2, 'Eswatini': 2, 'Togo': 2, 'Guinea-Bissau': 2, 'Mozambique': 2, 'Comoros': 2, 'Djibouti': 2, 'Namibia': 2, 'Somalia': 1, 'Ethiopia': 1}"
5,"World Cup Qualifiers, Africa",Preliminary Round 1,2015-10-05,2015-10-13,"{'Malawi': 2, 'Tanzania': 2, 'Ethiopia': 2, 'São Tomé and Príncipe': 2, 'Kenya': 2, 'Mauritius': 2, 'Madagascar': 2, 'Central African Republic': 2, 'Burundi': 2, 'Seychelles': 2, 'The Gambia': 2, 'Namibia': 2, 'Sierra Leone': 2, 'Chad': 2, 'Niger': 2, 'Somalia': 2, 'Lesotho': 2, 'Comoros': 2, 'Guinea-Bissau': 2, 'Liberia': 2, 'Mauritania': 2, 'South Sudan': 2, 'Botswana': 2, 'Eritrea': 2, 'Djibouti': 1, 'Eswatini': 1}"
6,"World Cup Qualifiers, Africa",Group Stage,2016-10-07,2017-11-14,"{'DR Congo': 6, 'Libya': 6, 'Tunisia': 6, 'Guinea': 6, 'Zambia': 6, 'Nigeria': 6, 'Algeria': 6, 'Cameroon': 6, 'Gabon': 6, 'Morocco': 6, 'Ivory Coast': 6, 'Mali': 6, 'Burkina Faso': 6, 'South Africa': 7, 'Senegal': 7, 'Cape Verde': 6, 'Ghana': 6, 'Uganda': 6, 'Congo': 6, 'Egypt': 6}"
7,"World Cup Qualifiers, Africa",Round 3,2017-08-31,2017-11-14,"{'Guinea': 4, 'Libya': 4, 'Tunisia': 4, 'DR Congo': 4, 'Nigeria': 4, 'Cameroon': 4, 'Zambia': 4, 'Algeria': 4, 'Morocco': 4, 'Mali': 4, 'Gabon': 4, 'Ivory Coast': 4, 'South Africa': 5, 'Senegal': 5, 'Cape Verde': 4, 'Burkina Faso': 4, 'Uganda': 4, 'Egypt': 4, 'Ghana': 4, 'Congo': 4}"
8,"World Cup Qualifiers, CONCACAF",Preliminary Round,2011-06-15,2011-07-16,"{'Bahamas': 2, 'Turks and Caicos Iss': 2, 'Dominican Republic': 2, 'Anguilla': 2, 'US Virgin Islands': 2, 'British Virgin Islands': 2, 'Saint Lucia': 2, 'Aruba': 2, 'Montserrat': 1, 'Belize': 1}"
9,"World Cup Qualifiers, CONCACAF",Group Stage,2011-09-02,2011-11-15,"{'Suriname': 5, 'Cayman Islands': 6, 'El Salvador': 5, 'Dominican Republic': 6, 'Trinidad and Tobago': 6, 'Bermuda': 6, 'Guyana': 6, 'Barbados

In [134]:
merged_df = pd.merge(multiple_match_counts_df, team_appearances_df, on=["Competition", "Stage", "Start Date", "End Date"])
pd.set_option('display.max_colwidth', None)

In [135]:
merged_df['Case'] = 'Multiple Counts'

In [136]:
# Merge merged_df with results_df
final_df = pd.merge(results_df, merged_df, on=["Competition", "Stage", "Start Date", "End Date", 'Match Counts'], how='left', suffixes=('', '_merged'))

# Display the final DataFrame
final_df

,Competition,Stage,Start Date,End Date,Match Counts,Team Appearances,Case
0,"World Cup Qualifiers, Asia",Preliminary Round 1,2011-06-29,2011-07-03,2,NaN,NaN
1,"World Cup Qualifiers, Asia",Preliminary Round 2,2011-07-23,2011-07-28,2,NaN,NaN
2,"World Cup Qualifiers, Asia",Group Stage,2011-09-02,2012-02-29,6,NaN,NaN
3,"World Cup Qualifiers, Asia",Round 3,2012-06-03,2013-06-18,8,NaN,NaN
4,"World Cup Qualifiers, Asia",Round 4,2013-09-06,2014-06-18,2,NaN,NaN
...,...,...,...,...,...,...,...
74,Africa Cup of Nations,Group Stage,2013-01-19,2013-01-29,,NaN,NaN
75,Africa Cup of Nations,Group Stage,2015-01-17,2015-01-28,3,NaN,NaN
76,Africa Cup of Nations,Group Stage,2017-01-14,2017-01-25,3,NaN,NaN
77,Africa Cup of Nations,Group Stage,2019-06-21,2019-07-02,3,NaN,NaN


In [137]:
# Set the "Case" column to "NO MATCHES" for rows where "Match Counts" is an empty string
final_df.loc[final_df['Match Counts'] == '', 'Case'] = 'NO MATCHES'

In [138]:
final_df.loc[final_df['Match Counts'] == '', 'Match Counts'] = 0

In [139]:
final_df[final_df['Match Counts'] == 0]

,Competition,Stage,Start Date,End Date,Match Counts,Team Appearances,Case
35,"World Cup Qualifiers, CONCACAF",Round 3,2022-06-04,2022-06-08,0,NaN,NO MATCHES
45,"Euros, Qualifiers",Qualifying Round,2010-08-11,2011-11-15,0,NaN,NO MATCHES
48,Euros,Group Stage,2012-06-08,2012-06-18,0,NaN,NO MATCHES
51,Copa America,Group Stage,2011-07-01,2011-07-13,0,NaN,NO MATCHES
56,Olympics,Group Stage,2012-07-26,2012-08-01,0,NaN,NO MATCHES
61,Gold Cup,Group Stage,2011-06-05,2011-06-13,0,NaN,NO MATCHES
67,"Africa Cup of Nations, Qualifiers",Qualifying Round,2010-07-01,2011-10-09,0,NaN,NO MATCHES
68,"Africa Cup of Nations, Qualifiers",Qualifying Round,2012-01-01,2012-10-14,0,NaN,NO MATCHES
73,Africa Cup of Nations,Group Stage,2012-01-21,2012-01-31,0,NaN,NO MATCHES
74,Africa Cup of Nations,Group Stage,2013-01-19,2013-01-29,0,NaN,NO MATCHES


In [140]:
final_df.loc[~final_df['Case'].isin(['Multiple Counts', 'NO MATCHES']), 'Case'] = 'Single Total'

In [144]:
import pandas as pd

# Define the function to count unique teams involved
def count_unique_teams(latest_match_data, competition, start_date, end_date):
    # Convert string dates to datetime objects
    start_date_obj = pd.to_datetime(start_date)
    end_date_obj = pd.to_datetime(end_date)

    # Convert the 'Date' column in latest_match_data to datetime objects
    latest_match_data['Date'] = pd.to_datetime(latest_match_data['Date'])

    # Filter latest_match_data
    filtered_matches = latest_match_data[
        (latest_match_data['Competition'] == competition) &
        (latest_match_data['Date'] >= start_date_obj) &
        (latest_match_data['Date'] <= end_date_obj)
    ]

    # Get unique teams involved
    unique_teams = set(filtered_matches['Team.1.Name']).union(set(filtered_matches['Team.2.Name']))

    return len(unique_teams)

# Apply the function to each row of final_df
final_df['Unique Teams'] = final_df.apply(
    lambda row: count_unique_teams(latest_match_data, row['Competition'], row['Start Date'], row['End Date']),
    axis=1
)

# Display the updated DataFrame

In [147]:
final_df[final_df['Case'] == 'Multiple Counts']

,Competition,Stage,Start Date,End Date,Match Counts,Team Appearances,Case,Unique Teams
6,"World Cup Qualifiers, Asia",Preliminary Round 2,2015-06-11,2016-03-29,"8, 5, 6, 7","{'Iraq': 6, 'Chinese Taipei': 6, 'Yemen': 8, 'Uzbekistan': 8, 'Bahrain': 8, 'Kuwait': 5, 'Lebanon': 8, 'Myanmar': 7, 'Malaysia': 8, 'Timor-Leste': 8, 'Saudi Arabia': 8, 'Palestine': 8, 'United Arab Emirates': 8, 'Bangladesh': 8, 'Kyrgyzstan': 8, 'Tajikistan': 8, 'Jordan': 8, 'Australia': 8, 'Hong Kong': 8, 'Bhutan': 8, 'Maldives': 8, 'Qatar': 8, 'China': 8, 'Guam': 8, 'Turkmenistan': 8, 'India': 8, 'Oman': 8, 'Iran': 8, 'Cambodia': 8, 'Singapore': 8, 'Afghanistan': 8, 'Syria': 8, 'Japan': 8, 'Thailand': 5, 'Vietnam': 5, 'Laos': 7, 'South Korea': 7, 'Philippines': 8, 'North Korea': 8}",Multiple Counts,39
8,"World Cup Qualifiers, Asia",Round 3,2017-09-05,2017-11-15,"1, 3","{'Syria': 3, 'Australia': 3, 'Iran': 1, 'Qatar': 1, 'China': 1, 'Uzbekistan': 1, 'South Korea': 1, 'Thailand': 1, 'Iraq': 1, 'United Arab Emirates': 1, 'Saudi Arabia': 1, 'Japan': 1}",Multiple Counts,12
9,"World Cup Qualifiers, Asia",Preliminary Round 1,2019-06-06,2019-06-11,"1, 2","{'Guam': 2, 'Bhutan': 2, 'Brunei': 2, 'Mongolia': 2, 'Timor-Leste': 2, 'Malaysia': 2, 'Bangladesh': 2, 'Laos': 2, 'Pakistan': 2, 'Cambodia': 2, 'Macau': 1, 'Sri Lanka': 1}",Multiple Counts,12
10,"World Cup Qualifiers, Asia",Preliminary Round 2,2019-09-05,2021-06-15,"8, 5, 7","{'Guam': 8, 'Maldives': 8, 'Philippines': 8, 'Syria': 8, 'China': 8, 'Chinese Taipei': 8, 'Jordan': 8, 'Kuwait': 8, 'Nepal': 8, 'Australia': 8, 'Cambodia': 8, 'Hong Kong': 8, 'Bahrain': 8, 'Iraq': 8, 'Iran': 8, 'Singapore': 8, 'Yemen': 8, 'Palestine': 8, 'Uzbekistan': 8, 'Saudi Arabia': 8, 'India': 8, 'Oman': 8, 'Qatar': 8, 'Afghanistan': 8, 'Bangladesh': 8, 'Mongolia': 8, 'Myanmar': 8, 'Tajikistan': 8, 'Kyrgyzstan': 8, 'Japan': 8, 'Thailand': 8, 'Vietnam': 8, 'Indonesia': 8, 'Malaysia': 8, 'United Arab Emirates': 8, 'North Korea': 5, 'Lebanon': 8, 'Sri Lanka': 7, 'Turkmenistan': 7, 'South Korea': 7}",Multiple Counts,40
13,"World Cup Qualifiers, Africa",Preliminary Round,2011-11-11,2011-11-15,"1, 2","{'Burundi': 2, 'Lesotho': 2, 'Madagascar': 2, 'Equatorial Guinea': 2, 'Tanzania': 2, 'Chad': 2, 'Kenya': 2, 'Seychelles': 2, 'Rwanda': 2, 'Eritrea': 2, 'Congo': 2, 'São Tomé and Príncipe': 2, 'DR Congo': 2, 'Eswatini': 2, 'Togo': 2, 'Guinea-Bissau': 2, 'Mozambique': 2, 'Comoros': 2, 'Djibouti': 2, 'Namibia': 2, 'Somalia': 1, 'Ethiopia': 1}",Multiple Counts,22
16,"World Cup Qualifiers, Africa",Preliminary Round 1,2015-10-05,2015-10-13,"1, 2","{'Malawi': 2, 'Tanzania': 2, 'Ethiopia': 2, 'São Tomé and Príncipe': 2, 'Kenya': 2, 'Mauritius': 2, 'Madagascar': 2, 'Central African Republic': 2, 'Burundi': 2, 'Seychelles': 2, 'The Gambia': 2, 'Namibia': 2, 'Sierra Leone': 2, 'Chad': 2, 'Niger': 2, 'Somalia': 2, 'Lesotho': 2, 'Comoros': 2, 'Guinea-Bissau': 2, 'Liberia': 2, 'Mauritania': 2, 'South Sudan': 2, 'Botswana': 2, 'Eritrea': 2, 'Djibouti': 1, 'Eswatini': 1}",Multiple Counts,26
18,"World Cup Qualifiers, Africa",Group Stage,2016-10-07,2017-11-14,"6, 7","{'DR Congo': 6, 'Libya': 6, 'Tunisia': 6, 'Guinea': 6, 'Zambia': 6, 'Nigeria': 6, 'Algeria': 6, 'Cameroon': 6, 'Gabon': 6, 'Morocco': 6, 'Ivory Coast': 6, 'Mali': 6, 'Burkina Faso': 6, 'South Africa': 7, 'Senegal': 7, 'Cape Verde': 6, 'Ghana': 6, 'Uganda': 6, 'Congo': 6, 'Egypt': 6}",Multiple Counts,20
19,"World Cup Qualifiers, Africa",Round 3,2017-08-31,2017-11-14,"4, 5","{'Guinea': 4, 'Libya': 4, 'Tunisia': 4, 'DR Congo': 4, 'Nigeria': 4, 'Cameroon': 4, 'Zambia': 4, 'Algeria': 4, 'Morocco': 4, 'Mali': 4, 'Gabon': 4, 'Ivory Coast': 4, 'South Africa': 5, 'Senegal': 5, 'Cape Verde': 4, 'Burkina Faso': 4, 'Uganda': 4, 'Egypt': 4, 'Ghana': 4, 'Congo': 4}",Multiple Counts,20
23,"World Cup Qualifiers, CONCACAF",Preliminary Round,2011-06-15,2011-07-16,"1, 2","{'Bahamas': 2, 'Turks and Caicos Iss': 2, 'Dominican Republic': 2, 'Anguilla': 2, 'US Virgin Islands': 2, 'British Virgin Islands': 2, 'Saint Lucia

In [146]:
final_df[final_df['Case'] == 'Single Total']

,Competition,Stage,Start Date,End Date,Match Counts,Team Appearances,Case,Unique Teams
0,"World Cup Qualifiers, Asia",Preliminary Round 1,2011-06-29,2011-07-03,2,NaN,Single Total,16
1,"World Cup Qualifiers, Asia",Preliminary Round 2,2011-07-23,2011-07-28,2,NaN,Single Total,30
2,"World Cup Qualifiers, Asia",Group Stage,2011-09-02,2012-02-29,6,NaN,Single Total,20
3,"World Cup Qualifiers, Asia",Round 3,2012-06-03,2013-06-18,8,NaN,Single Total,10
4,"World Cup Qualifiers, Asia",Round 4,2013-09-06,2014-06-18,2,NaN,Single Total,2
5,"World Cup Qualifiers, Asia",Preliminary Round 1,2015-03-12,2015-03-17,2,NaN,Single Total,12
7,"World Cup Qualifiers, Asia",Group Stage,2016-06-07,2017-06-13,8,NaN,Single Total,12
11,"World Cup Qualifiers, Asia",Group Stage,2021-09-02,2022-03-29,10,NaN,Single Total,12
12,"World Cup Qualifiers, Asia",Round 3,2022-06-07,2022-06-14,1,NaN,Single Total,2
14,"World Cup Qualifiers, Africa",Group Stage,2012-06-01,2013-09-10,6,NaN,Single Total,40


In [155]:
final_df[final_df['Competition'] == 'World Cup Qualifiers, Africa']

,Competition,Stage,Start Date,End Date,Match Counts,Team Appearances,Case,Unique Teams
13,"World Cup Qualifiers, Africa",Preliminary Round,2011-11-11,2011-11-15,"1, 2","{'Burundi': 2, 'Lesotho': 2, 'Madagascar': 2, 'Equatorial Guinea': 2, 'Tanzania': 2, 'Chad': 2, 'Kenya': 2, 'Seychelles': 2, 'Rwanda': 2, 'Eritrea': 2, 'Congo': 2, 'São Tomé and Príncipe': 2, 'DR Congo': 2, 'Eswatini': 2, 'Togo': 2, 'Guinea-Bissau': 2, 'Mozambique': 2, 'Comoros': 2, 'Djibouti': 2, 'Namibia': 2, 'Somalia': 1, 'Ethiopia': 1}",Multiple Counts,22
14,"World Cup Qualifiers, Africa",Group Stage,2012-06-01,2013-09-10,6,NaN,Single Total,40
15,"World Cup Qualifiers, Africa",Round 3,2013-10-11,2013-11-19,2,NaN,Single Total,10
16,"World Cup Qualifiers, Africa",Preliminary Round 1,2015-10-05,2015-10-13,"1, 2","{'Malawi': 2, 'Tanzania': 2, 'Ethiopia': 2, 'São Tomé and Príncipe': 2, 'Kenya': 2, 'Mauritius': 2, 'Madagascar': 2, 'Central African Republic': 2, 'Burundi': 2, 'Seychelles': 2, 'The Gambia': 2, 'Namibia': 2, 'Sierra Leone': 2, 'Chad': 2, 'Niger': 2, 'Somalia': 2, 'Lesotho': 2, 'Comoros': 2, 'Guinea-Bissau': 2, 'Liberia': 2, 'Mauritania': 2, 'South Sudan': 2, 'Botswana': 2, 'Eritrea': 2, 'Djibouti': 1, 'Eswatini': 1}",Multiple Counts,26
17,"World Cup Qualifiers, Africa",Preliminary Round 2,2015-11-09,2015-11-17,2,NaN,Single Total,40
18,"World Cup Qualifiers, Africa",Group Stage,2016-10-07,2017-11-14,"6, 7","{'DR Congo': 6, 'Libya': 6, 'Tunisia': 6, 'Guinea': 6, 'Zambia': 6, 'Nigeria': 6, 'Algeria': 6, 'Cameroon': 6, 'Gabon': 6, 'Morocco': 6, 'Ivory Coast': 6, 'Mali': 6, 'Burkina Faso': 6, 'South Africa': 7, 'Senegal': 7, 'Cape Verde': 6, 'Ghana': 6, 'Uganda': 6, 'Congo': 6, 'Egypt': 6}",Multiple Counts,20
19,"World Cup Qualifiers, Africa",Round 3,2017-08-31,2017-11-14,"4, 5","{'Guinea': 4, 'Libya': 4, 'Tunisia': 4, 'DR Congo': 4, 'Nigeria': 4, 'Cameroon': 4, 'Zambia': 4, 'Algeria': 4, 'Morocco': 4, 'Mali': 4, 'Gabon': 4, 'Ivory Coast': 4, 'South Africa': 5, 'Senegal': 5, 'Cape Verde': 4, 'Burkina Faso': 4, 'Uganda': 4, 'Egypt': 4, 'Ghana': 4, 'Congo': 4}",Multiple Counts,20
20,"World Cup Qualifiers, Africa",Preliminary Round 1,2019-09-04,2019-09-10,2,NaN,Single Total,28
21,"World Cup Qualifiers, Africa",Group Stage,2020-03-22,2021-11-16,6,NaN,Single Total,40
22,"World Cup Qualifiers, Africa",Round 3,2022-03-24,2022-03-29,2,NaN,Single Total,10


In [ ]:
#TO DO

    #WC Qualifiers CONCACAF - figure out are the dates a tiny bit off? 
        #why does it say Round 4 for 2014 and 2018 is 9 matches not 10? 
        #why is 2018 group stage 5 matches not 6
        #why is 2014 round 3 mostly 5 matches not 6

        #why is 2022 group stage 13 matches not 14? 

        #Change the date of Prelim round 1 for 2022 from 2021-03-30 to 2021-06-08. that will include more matches fs

    #WCQ Europe
        #2014 
            #Look for matches between 14 and 20 November 2013. this is the PLAYOFF ROUND. ALL SURVIVAL MODE

        #2018
            #Look for matches between 11 and 15 November 2017. this is the PLAYOFF ROUND. ALL SURVIVAL MODE

        #2022
            #Look for matches between March and July 2022. this is the PLAYOFF ROUND. ALL SURVIVAL MODE


    #WCQ Asia 
        #### DONE – for 2018 Prelim round 2, why does the # of matches change so much for teams?
            #Kuwait and Indonesia got DQ'd.
        #### DONE - 2022 prelim rd 2 - why the # of matches changing for 2022 group stage??
            #NK was DQ'd

        #For 2018 group stage, change date from 2017-06-13	to 2017-09-05 

    #WCQ Africa
        #REMOVE THE MATCH FROM SEPTEMBER 5 or 6 2017 WHERE SA BEATS SENEGAL 2-1
        #"round 3" doesn't exist. delete that. 
        #Group stage dates should be 2016-10-03 to 2017-11-14.


In [ ]:
#RULES BY COMPETITION

    #WC Qualifiers Africa 
        #2014
            #Preliminary Round - ALL SURVIVAL MODE
            #Group stage - only the group winners advanced. (10 groups of 4)
            #Round 3 - ALL SURVIVAL MODE - 2-LEGGED TIES. 
        #2018
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Preliminary Round 2 - ALL SURVIVAL MODE
            #Group Stage / Round 3 - only the group winners advanced. (5 groups of 4) 
        #2022
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Group stage - only the group winners advanced. (10 groups of 4)
            #Round 3 - ALL SURVIVAL MODE
    #WC Qualifiers Asia
        #2014
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Preliminary Round 2 - ALL SURVIVAL MODE
            #Group Stage - top 2 teams advance (5 groups of 4)
            #Round 3 - top 3 teams advance (2 groups of 5)
            #Round 4 - ALL SURVIVAL MODE
        #2018
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Preliminary Round 2 - 8 group winners & top 4 2nd place teams go through(8 groups of 5)
            #Group Stage - top 3 teams advance (2 groups of 6)
            #Round 3 - ALL SURVIVAL MODE
        #2022
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Preliminary Round 2 - 8 group winners & top 4 2nd place teams go through(8 groups of 5)
            #Group Stage - top 3 teams advance (2 groups of 6)
            #Round 3 - ALL SURVIVAL MODE

    #WC Qualifiers CONCACAF
        #2014
            #Preliminary Round - ALL SURVIVAL MODE
            #Group Stage - top team advances (6 groups of 4)
            #Round 3 - top 2 in each group advance (3 groups of 4)
            #Round 4 - top 4 teams go through (1 group of 6)
        #2018
            #Preliminary Round 1 - ALL SURVIVAL MODE
            #Preliminary Round 2 - ALL SURVIVAL MODE
            #Preliminary Round 3 - ALL SURVIVAL MODE
            #Group Stage - top 2 teams go through (3 groups of 4)
            #Round 4 - top 4 teams go through (1 group of 6)
        #2022
            #Preliminary Round 1 - only the top team goes through (6 groups of 5)
            #Preliminary Round 2 - SURVIVAL MODE
            #Group Stage - top 4 teams go through (1 group of 8)
    #WC Qualifiers CONMEBOL
        #All years - Top 5 teams go through (1 group of 10)
    #WC Qualifiers UEFA
        #2014
            #Group Stage - the top team and the top 8 of 9 2nd place teams advance (8 groups of 6, 1 group of 5)
        #2018
            #Group Stage - the top team and the top 8 of 9 2nd place teams advance (9 groups of 6)
        #2022
            #Group Stage - the top 2 teams advance (5 groups of 6, 5 groups of 5)


In [ ]:
#3 sets of rules

    #1. where it works - all played X games
        #you can apply to this group. the # match days calculation thing
            #all the finals (gold cup, copa america, euros, world cup, afcon)

    #2. DIFFERENT NUMBERS OF GAMES 
        #all the WC qualifiers for each country - DONE

        #EUROS QUALIFIERS - TO DO 
        #AFCON QUALIFIERS - TO DO 

    #3. NO GAMES - IGNORE.
        #competitions/matches we do not have 
            #AFCON 2012 - GOT IT
                #We just didnt get that
            #AFCON 2013 - GOT IT
                #We just didnt get that
            #AFCON Qualifiers 2010 - GOT IT
                #We just didnt get that
            #AFCON Qualifiers 2012 - GOT IT
                #We just didnt get that
            #Gold Cup 2011 - GOT IT
                #We just didnt get that
            #Olympics 2012 - GOT IT
                #We just didnt get that
            #Copa America 2011 - GOT IT
                #We just didnt get that
            #Euros Qualifiers 2010 - GOT IT
                #We just didnt get that
            #CONCACAF WC Qualifiers 2022 - GOT IT
                #they were done by 2021
            #Euros 2012 - GOT IT
                #We just didnt get that

